In [ ]:
!pip install google-api-python-client

INFO:py4j.clientserver:Received command c on object id p0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from datetime import datetime
import pandas as pd
from io import StringIO
from googleapiclient.discovery import build
from pandas import DataFrame

INFO:py4j.clientserver:Received command c on object id p1


In [ ]:
def coletar_dados_youtube(api_key: str, max_results = 50) -> list:
    """Coletando dados do YouTube"""
    youtube = build('youtube', 'v3', developerKey = api_key)

    search_response = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        chart='mostPopular',
        regionCode='BR',
        maxResults=max_results
    ).execute()

    dados = [{'dados': video} for video in search_response.get("items", [])]
    return dados

INFO:py4j.clientserver:Received command c on object id p1


In [ ]:
def criar_dataframe(dados: list) -> DataFrame:
    """Criando DataFrame"""
    return pd.DataFrame(dados)

INFO:py4j.clientserver:Received command c on object id p1


In [ ]:
def verificar_excluir_arquivo_blob(blob_client, nome_arquivo: str) -> None:
    """Verificando e excluindo arquivo existente no Blob Storage"""
    if blob_client.exists():
        blob_client.delete_blob()

INFO:py4j.clientserver:Received command c on object id p1


In [ ]:
def fazer_upload_blob(blob_client, nome_arquivo : str, conteudo) -> None:
    """Fazendo upload do arquivo para o Blob Storage"""
    blob_client.upload_blob(StringIO(conteudo).read().encode('utf-8'))

INFO:py4j.clientserver:Received command c on object id p1


In [ ]:
if __name__ == "__main__":
    
    # Configurações
    key = ''
    nome_conta = ''
    key_conta = ''
    nome_container = "bronze"
    nome_arquivo = 'data_' + datetime.now().strftime("%Y%m%d") + '.csv'

    # Conectar ao Blob Storage Bronze
    url_blob_storage = f"https://{nome_conta}.blob.core.windows.net"
    blob_service_client = BlobServiceClient(account_url=url_blob_storage, credential=key_conta)
    container_client = blob_service_client.get_container_client(nome_container)
    blob_client = container_client.get_blob_client(nome_arquivo)

    # Coletar dados
    dados = coletar_dados_youtube(key)

    #Criar DataFrame
    df = criar_dataframe(dados)
    
    # Verificar/excluir arquivo existente
    verificar_excluir_arquivo_blob(blob_client, nome_arquivo)
    
    # Fazer upload do novo arquivo
    new_content = df.to_csv(index=False)
    fazer_upload_blob(blob_client, nome_arquivo, new_content)


INFO:py4j.clientserver:Received command c on object id p1
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:py4j.clientserver:Received command c on object id p0
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://etldatarafael.blob.core.windows.net/bronze/data_20240125.csv'
Request method: 'HEAD'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.19.0 Python/3.10.12 (Linux-5.15.0-1053-azure-x86_64-with-glibc2.35)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '7cbb7ee4-bb81-11ee-84f4-00163ef8255f'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '150976'
    'Content-Type': 'application/octet-stream'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Thu, 25 Jan 2024 12:17:04 GMT'
    'Accept-Range